In [17]:
!git status

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   Collab_func.py
	modified:   Collect_data.py

no changes added to commit (use "git add" and/or "git commit -a")


In [4]:
%run "Collab_func.py"
%run "Collect_data.py"
%run "bootstrapped_ways"
%run "MAB.py"

In [ ]:
# info_experiments_empty = {}
# with open('/content/drive/MyDrive/AB_testing/' + file_name + '.json', 'w') as f:
#   json.dump(info_experiments_empty, f)

In [ ]:
file_name = "rev_hist_dict_7"
def ucb_boot_testing(rev_hist_dict: Dict[str, np.array], 
                     file_name: str,
                     number_exp: int=1000, 
                     number_boots: int=10000, 
                     new_observations: int=1
                     ):
    """
    Testing bootstrapped UCB method for robustness
    rev_hist_dict: historical revenue values
    number_exp: number of new users 
    number_boots: number of bootstrapped samples
    new_observations: number of users we include to recalculate bounders

    return: json files with results 
    """
    all_values = [values for key, values in rev_hist_dict.items()]
    revenue_all = np.array(sum(all_values, [])).astype(np.float32)
    rev_exp_dict = {key: revenue_all for key in rev_hist_dict.keys()}
    n_exp = number_exp // new_observations
    info_exp = {"last_iter_time": [],
                "number_exp": number_exp,
                "number_boots": number_boots,
                "new_observations": new_observations}
    start_experiment = datetime.now().strftime("%Y%m%d%H%M%S.%f")

    with open('/content/drive/MyDrive/AB_testing/' + file_name + '.json') as f:
        data = json.load(f)
    info_experiments = {start_experiment: {'info': info_exp,
                             'revenue_hist': rev_hist_dict,
                             "revenue_experiment": {
                                     "actions": [],
                                  "revenues": []
                             }}
                        }
    data.update(info_experiments)
    write_json(data, '/content/drive/MyDrive/AB_testing/' + file_name + '.json')
    for i in tqdm(range(n_exp)):
        with open('/content/drive/MyDrive/AB_testing/' + file_name + '.json') as f:
            revenue_experiment = json.load(f)
        n_action = ucb_bootstrapped(rev_exp_dict, [number_boots] * len(rev_exp_dict.keys()))
        random_revenues = [gener_random_revenue(rev_hist_dict[n_action])[0] for i in range(new_observations)]
        rev_exp_dict[n_action] = np.append(rev_exp_dict[n_action], random_revenues)
      # Write results in json file
        revenue_experiment[start_experiment]['revenue_experiment']['revenues'].append(random_revenues)
        revenue_experiment[start_experiment]['revenue_experiment']['actions'].append(n_action)
        last_iter_time = datetime.now().strftime("%Y%m%d%H%M%S.%f")
        revenue_experiment[start_experiment]['info']['last_iter_time'] = last_iter_time
        write_json(revenue_experiment, '/content/drive/MyDrive/AB_testing/' + file_name + '.json')
    return
for n_boots in tqdm([10000, 20000, 30000, 40000, 50000]):
    for number_exp in tqdm([2500, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]):
        for new_obs in tqdm([1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]):
            ucb_boot_testing(rev_hist_dict=rev_hist_dict_7,
                       file_name=file_name, 
                       number_exp=number_exp, 
                       number_boots=n_boots, 
                       new_observations=new_obs)

# exp_rev = ucb_boot_testing(rev_hist_dict_7, number_exp=50)

/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:365: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)
